In [1]:
import glob
import os
import re
import pandas as pd
import numpy as np

In [2]:
print os.getcwd()
os.chdir("/Users/palermospenano/Desktop/github_proj/ejmr_analysis/ejmr_data")

/Users/palermospenano/Desktop/github_proj/ejmr_analysis


In [3]:
# helps sort file by number at the end of each filename (10 correctly placed after 9 in filename)
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [4]:
# concatenate all csv files
ejmr_data = sorted(glob.glob('ejmr_pg*.csv'), key=numericalSort)

df_list = []
for filename in sorted(ejmr_data):
    df_list.append(pd.read_csv(filename))
full_df = pd.concat(df_list)

In [5]:
# split title into orig. link and post title, and get freshness values
full_df['post_link'] = full_df['title'].str.extract(r'<a href="(.*?)">',expand=True)
full_df['post_title'] = full_df['title'].str.extract(r'">(.*?)</a>',expand=True)
full_df['how_fresh'] = full_df['freshness'].str.extract(r'">(.*?)</a>',expand=True)

list(full_df.columns)

# reorder columns
full_df = full_df[['post_title',
                   'num_posts',
                   'views',
                   'votes',
                   'how_fresh',
                   'download_time',
                   'title',
                   'post_link',
                   'freshness'
                  ]]

# export full data to csv
full_df.to_csv('ejmr_data.csv', index=True, index_label='pandas_index')

In [6]:
full_df.head()

,post_title,num_posts,views,votes,how_fresh,download_time,title,post_link,freshness
0,Pls explain me how machines can learn?,5,95,0-0,1 day,2017-06-11 03:37:22.777761,"<a href=""https://www.econjobrumors.com/topic/p...",https://www.econjobrumors.com/topic/pls-explai...,"<a href=""https://www.econjobrumors.com/topic/p..."
1,Machine learning is Satan worshipping in disguise,17,337,6-0,1 day,2017-06-11 03:37:22.777852,"<a href=""https://www.econjobrumors.com/topic/m...",https://www.econjobrumors.com/topic/machine-le...,"<a href=""https://www.econjobrumors.com/topic/m..."
2,Should I buy a BMW,29,960,0-0,1 day,2017-06-11 03:37:22.777956,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/should-i-b...,"<a href=""https://www.econjobrumors.com/topic/s..."
3,A proposal on how to replace student evaluations,22,561,4-3,1 day,2017-06-11 03:37:22.778058,"<a href=""https://www.econjobrumors.com/topic/a...",https://www.econjobrumors.com/topic/a-proposal...,"<a href=""https://www.econjobrumors.com/topic/a..."
4,"Roman Catholics, put down your rosaries and ex...",1,69,1-0,1 day,2017-06-11 03:37:22.778133,"<a href=""https://www.econjobrumors.com/topic/r...",https://www.econjobrumors.com/topic/roman-cath...,"<a href=""https://www.econjobrumors.com/topic/r..."


In [7]:
ejmr_df = pd.read_csv('ejmr_data.csv')

In [8]:
def nonNaN_title_only(df, df_col):
    """
    replaces empty strings with NaN and subsequently removes removes them. Function makes changes in place.
    """
    print "Total number of rows: {0}".format(len(df))
    print len(df) - df.count()  # number of NaN rows for each column
    
    df[df_col].replace('', np.nan, inplace=True)
    df.dropna(subset=[df_col], inplace=True)
    
    print "Total number of rows after deleting NaN: {0}".format(len(df))
    
    return df

def cleanText(text):
    """
    removes punctuation, stopwords and returns lowercase text in a list of single words
    source: http://francescopochetti.com/tag/sentiment-analysis/
    """
    text = text.lower()    

    from bs4 import BeautifulSoup
    text = BeautifulSoup(text).get_text()

    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)

    from nltk.corpus import stopwords
    clean = [word for word in text if word not in stopwords.words('english')]

    return clean

In [9]:
nonNaN_title_only(ejmr_df,'post_title')

Total number of rows: 288501
pandas_index        0
post_title       2385
num_posts           0
views               0
votes               0
how_fresh           0
download_time       0
title               0
post_link        2347
freshness           0
dtype: int64
Total number of rows after deleting NaN: 286116


,pandas_index,post_title,num_posts,views,votes,how_fresh,download_time,title,post_link,freshness
0,0,Pls explain me how machines can learn?,5,95,0-0,1 day,2017-06-11 03:37:22.777761,"<a href=""https://www.econjobrumors.com/topic/p...",https://www.econjobrumors.com/topic/pls-explai...,"<a href=""https://www.econjobrumors.com/topic/p..."
1,1,Machine learning is Satan worshipping in disguise,17,337,6-0,1 day,2017-06-11 03:37:22.777852,"<a href=""https://www.econjobrumors.com/topic/m...",https://www.econjobrumors.com/topic/machine-le...,"<a href=""https://www.econjobrumors.com/topic/m..."
2,2,Should I buy a BMW,29,960,0-0,1 day,2017-06-11 03:37:22.777956,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/should-i-b...,"<a href=""https://www.econjobrumors.com/topic/s..."
3,3,A proposal on how to replace student evaluations,22,561,4-3,1 day,2017-06-11 03:37:22.778058,"<a href=""https://www.econjobrumors.com/topic/a...",https://www.econjobrumors.com/topic/a-proposal...,"<a href=""https://www.econjobrumors.com/topic/a..."
4,4,"Roman Catholics, put down your rosaries and ex...",1,69,1-0,1 day,2017-06-11 03:37:22.778133,"<a href=""https://www.econjobrumors.com/topic/r...",https://www.econjobrumors.com/topic/roman-cath...,"<a href=""https://www.econjobrumors.com/topic/r..."
5,5,Eh...eh...brains....aARGHHH BRAINS,2,63,0-0,1 day,2017-06-11 03:37:22.778206,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ehehbrains...,"<a href=""https://www.econjobrumors.com/topic/e..."
6,6,Taylor Swift puts all music on Spotify on Katy...,7,365,1-1,1 day,2017-06-11 03:37:22.778279,"<a href=""https://www.econjobrumors.com/topic/t...",https://www.econjobrumors.com/topic/taylor-swi...,"<a href=""https://www.econjobrumors.com/topic/t..."
7,7,2016 Maryland Senior Hires,104,10160,0-0,1 day,2017-06-11 03:37:22.778472,"<a href=""https://www.econjobrumors.com/topic/2...",https://www.econjobrumors.com/topic/2016-maryl...,"<a href=""https://www.econjobrumors.com/topic/2..."
8,8,Seminar Notice from CCER,4,392,0-0,1 day,2017-06-11 03:37:22.778543,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/seminar-no...,"<a href=""https://www.econjobrumors.com/topic/s..."
9,9,EJMR Rule #1: Be Respectful,71,783,3-7,1 day,2017-06-11 03:37:22.778686,"<a href=""https://www.econjobrumors.com/topic/e...",https://www.econjobrumors.com/topic/ejmr-rule-...,"<a href=""https://www.econjobrumors.com/topic/e..."


In [ ]:
# clean text (warning: takes a while)
ejmr_df['cleanTitle'] = ejmr_df['post_title'].apply(cleanText)

/Users/palermospenano/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/Users/palermospenano/anaconda/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://kirkland.econjobrumors.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.'

In [ ]:
# save cleaned data to csv
ejmr_df.to_csv('ejmr_data_cleand.csv', index=True, index_label='pandas_index')